In [ ]:
open System

// HELPER FUNCTIONS

let map, head, tail, fold, filter, toList, forall, exists, sum, except, rev = 
    List.map, List.head, List.tail, List.fold, List.filter, Seq.toList, List.forall, List.exists, List.sum, List.except, List.rev

let map2d f = map f |> map
let map3d f = map2d f |> map
let filter2d f = map (filter f)

let toInts (excps: string[]) (str: string) = 
    str.Split(excps, StringSplitOptions.RemoveEmptyEntries) |> toList |> map int

let rec transpose = function
| []::xs -> [] 
| xs -> map head xs :: transpose (map tail xs)

// DATA PREP

let rawRows = File.ReadAllLines "day4.txt" |> toList

let bingoNums = rawRows |> head |> toInts [|","|]

let foldCards cards = function
| row when String.IsNullOrEmpty row -> [] :: cards
| row -> ((toInts [| " "; "  "|] row) :: head cards) :: tail cards

let bingoCards =
    rawRows |> tail |> fold foldCards [] |> map3d (fun n -> (n, false))

// DOMAIN

let updateCell n1 (n2, b) = if n1 = n2 then (n1, true) else (n2, b)

let hasBingo = map (forall snd) >> exists id 

let cardResult newNum =
    filter2d (snd >> not) >> map2d fst >> map sum >> sum >> ((*) newNum)

let winningcards = filter (fun card -> hasBingo card || hasBingo (transpose card))

// PART 1
  
let rec bingoFinder (cards, nums) =
    let newNum = head nums
    let newcards = cards |> map3d (updateCell newNum)

    match winningcards newcards with
    | [] -> bingoFinder (newcards, tail nums)
    | winners -> head winners |> cardResult newNum  

let day4part1solution = bingoFinder (bingoCards, bingoNums)

// PART 2

let bingoFolder (cards, winners) num =
    let updatedCards = cards |> map3d (updateCell num)

    match winningcards updatedCards with
    | [] -> updatedCards, winners
    | newWinners ->
        let newCards = except newWinners updatedCards
        let winGroup = (num, rev newWinners)
        newCards, winGroup :: winners
    
let day4part2solution =
  fold bingoFolder (bingoCards, []) bingoNums |> snd |> head
  |> fun (bingoNumber, wingroup) -> cardResult bingoNumber (head wingroup)

day4part1solution, day4part2solution

Item1,Item2
10374,24742
